In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install nibabel fury dipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
  Created wheel for pygltflib: filename=pygltflib-1.16.1-py3-none-any.whl size=27085 sha256=c1a1d42a61d8b664e0083cc989d188f8e47abda8cb14e8ecd8898706cbabed45
  Stored in directory: /root/.cache/pip/wheels/9c/73/91/ae3a93bb6cc6dde662ed1d

In [ ]:
import numpy as np
from numpy import *
import matplotlib.pyplot as plt
import dipy.reconst.dki as dki
import dipy.reconst.dti as dti
from dipy.core.gradients import gradient_table
from dipy.data import get_fnames
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti
from dipy.segment.mask import median_otsu
from scipy.ndimage.filters import gaussian_filter
from dipy.io.image import load_nifti, save_nifti
import nibabel as nib
from multiprocessing import Process, Queue
import os
import numba as nb

<ipython-input-3-f466099a7c23>:11: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [ ]:
[X,Z,Y] = np.meshgrid(np.arange(-2,3),np.arange(-2,3),np.arange(-2,3))
XYZ = np.stack([X,Y,Z])
XYZ = XYZ.reshape(3,-1)
XYZ = XYZ[None,:,:]

In [ ]:
XYZ.shape

(1, 3, 125)

In [ ]:
def tf_subprocess(q):

  from tensorflow.keras.models import Sequential
  from tensorflow.keras.layers import Dense, Conv1D, Input, Reshape, InputLayer, Dropout,LayerNormalization,Flatten
  import numpy as np
  from tensorflow.keras import backend as kf
  import matplotlib.pyplot as plt
  import tensorflow as tf
  from tensorflow.keras import Model
  from tensorflow.keras.optimizers import Adam

  [jj,outer_currSz,bsize,lidx11,Image_diff_norm,bvec,y_val0,y_val1,y_val2,y_val3,y_val4,y_val5,yCount] = q.get()

  def rmse(yTrue, yPred):
        return kf.sqrt(kf.mean(10 * kf.pow(yTrue - yPred, 2)))

  def scaled_abs100(yTrue, yPred):
      return 1000 * 100 * kf.mean(kf.abs((yTrue - yPred)))

  def scaled_abs(yTrue, yPred):
      #wt = tf.constant([[1.0],[100.0],[100.0]])
      return 1000 * kf.mean(kf.abs((yTrue - yPred)))

  model= tf.keras.models.load_model(
      '21_diff_SwinTransformer_mode.tf', custom_objects={'scaled_abs': scaled_abs, 'scaled_abs100': scaled_abs100, 'rmse':rmse }, compile=True, options=None)

  for jj0 in range(0,outer_currSz, bsize):
    j0 = jj + jj0
    upLim = min(jj0+bsize,outer_currSz)
    currSz = upLim-jj0
    X = np.zeros((currSz, lidx11.shape[2],Image_diff_norm.shape[3]))
    for j1 in range(currSz):
      j = j1+j0
      for k in range(lidx11.shape[2]):
        coord = lidx11[j,:,k].reshape((3,))
        X[j1,k,:] = Image_diff_norm[coord[0],coord[1],coord[2],:]
        ##########################
      # x1FA = Image_diff_norm[idx1FA,:]
    #X = X.transpose((0,2,1))
    XX = np.zeros((currSz, 128, 100))
    for i in range(X.shape[0]):
      XX[i, :X.shape[1], :X.shape[2]] = X[i, :, :]
      XX[i, X.shape[1]:, :X.shape[2]] = bvec.T
    [yval0, yval1, yval2, yval3, yval4, yval5]= model.predict(XX,batch_size=2000) #batch_size=5
    for j1 in range(currSz):
      j = j1+j0
      for k in range(lidx11.shape[2]):
        coord = lidx11[j,:,k].reshape((3,))
        y_val0[coord[0],coord[1],coord[2]] += yval0[j1, k]
        y_val1[coord[0],coord[1],coord[2]] += yval1[j1, k]
        y_val2[coord[0],coord[1],coord[2]] += yval2[j1, k]
        y_val3[coord[0],coord[1],coord[2]] += yval3[j1, k]
        y_val4[coord[0],coord[1],coord[2]] += yval4[j1, k]
        y_val5[coord[0],coord[1],coord[2]] += yval5[j1, k]
        yCount[coord[0],coord[1],coord[2]] += 1

  #[yFA1, yAD1, yMD1]= model.predict(X,batch_size=batch_size)
  q.put([y_val0, y_val1, y_val2, y_val3, y_val4, y_val5, yCount])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Input, Reshape, InputLayer, Dropout,LayerNormalization,Flatten
import numpy as np
from tensorflow.keras import backend as kf
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam


def rmse(yTrue, yPred):
      return kf.sqrt(kf.mean(10 * kf.pow(yTrue - yPred, 2)))

def scaled_abs100(yTrue, yPred):
    return 1000 * 100 * kf.mean(kf.abs((yTrue - yPred)))

def scaled_abs(yTrue, yPred):
    #wt = tf.constant([[1.0],[100.0],[100.0]])
    return 1000 * kf.mean(kf.abs((yTrue - yPred)))

model= tf.keras.models.load_model(
    '21_diff_SwinTransformer_mode.tf', custom_objects={'scaled_abs': scaled_abs, 'scaled_abs100': scaled_abs100, 'rmse':rmse }, compile=True, options=None)


In [ ]:
param_path = '/content/drive/MyDrive/Ananya_Singhal_2010110087/ICIP - classification/AD_data_21diff'

folderpath_labels = '/content/drive/MyDrive/Ananya_Singhal_2010110087/ICIP - classification/AD_data_21diff/DTI_parameters_21_Diff_AD'


Testing = [
'005_S_5038_M_82_AD_AD',
'003_S_4152_M_61_AD_AD',
'005_S_4910_F_82_AD_AD',
'003_S_4892_F_75_AD_AD']

In [ ]:

for lst in Testing:
      file_path_test = os.path.join(param_path,'ordered_4d_image_21diff_{}.nii.gz'.format(lst, lst))

      Maskfilepath = os.path.join(folderpath_labels,lst,'tensor_mask_{}.nii.gz'.format(lst))

      for diff_count in [21]:    # 90, 64, 32, 21, 5


            #fullpath = folderpath
            fullpath= '/content/drive/MyDrive/Ananya_Singhal_2010110087/Code_Interpolation/'
            fbval1 = os.path.join(fullpath,'100206_bvals1000_21_Diff.bval')
            fbvec1 = os.path.join(fullpath,'100206_bvecs1000__21_Diff.bval')

            bvals, bvecs = read_bvals_bvecs(fbval1, fbvec1)

            data, affine = load_nifti(file_path_test) #loading ordered 4d test data
            mask, _ = load_nifti(Maskfilepath) #loading mask
            gtab = gradient_table(bvals, bvecs)

            print(data.shape)
            print(bvecs.shape)
            diffusionGradientList= bvecs
            noOfDiffusionComponent= bvecs.shape[0]

            diffusionBVALSList= bvals
            noOfBVALComponent= bvals.shape[0]

            norm_of_diffusionGradientList = diffusionGradientList.__pow__(2).sum(1).__pow__(0.5)

            # index_of_normalizing_bVecs = norm_of_diffusionGradientList < 1e-3
            index_of_normalizing_bVecs = np.logical_and(diffusionBVALSList >=0,diffusionBVALSList <= 10)
            indexList_diffGrad_B0 = np.arange(0, noOfDiffusionComponent)[index_of_normalizing_bVecs]
            indexList_diffGrad = np.arange(0, noOfDiffusionComponent)[~index_of_normalizing_bVecs]

            indexList_Bvals_B0 = np.arange(0, noOfBVALComponent)[index_of_normalizing_bVecs]
            indexList_Bvals = np.arange(0, noOfBVALComponent)[~index_of_normalizing_bVecs]

            bvec = bvecs[indexList_Bvals,:]
            bval = bvals[indexList_Bvals]

            print(bvec.shape, bval.shape)


            Image_total = np.double(data)
            Image_dim = Image_total.shape
            Image_0 = Image_total[:, :, :, indexList_diffGrad_B0]
            Image_diff = Image_total[:, :, :, indexList_diffGrad]
            Image_S0_div = Image_0.sum(3) / indexList_diffGrad_B0.size

            Image_S0_div_nonzeroIdx = np.reshape(np.array([Image_S0_div != 0]), Image_dim[0:3])

            Image_diff_norm = np.zeros(Image_diff.shape)
            Image_diff_norm[Image_S0_div_nonzeroIdx, :] = Image_diff[Image_S0_div_nonzeroIdx, :] / \
            Image_S0_div[Image_S0_div_nonzeroIdx][:, None]

            Image_diff_norm_gt_1 = Image_diff_norm > 1
            Image_diff_norm_lt_0 = Image_diff_norm < 0
            Image_diff_norm[Image_diff_norm_gt_1] = 1
            Image_diff_norm[Image_diff_norm_lt_0] = 0

            print(Image_diff_norm.shape)
            # X = Image_diff_norm.reshape((-1,Image_diff_norm.shape[3]))

            Image_diff_norm = np.pad(Image_diff_norm, ((2, 2), (2, 2), (2, 2), (0, 0)))
            mask = np.pad(mask, ((2, 2), (2, 2), (2, 2)))
            print(Image_diff_norm.shape)

            size = Image_diff_norm.shape

            mask_idx = (mask>0)
            mask_idx1 = mask_idx.reshape(-1)
            # [X1,Z1,Y1] = np.meshgrid(np.arange(size[0]),np.arange(size[1]),np.arange(size[2]))
            [X1,Y1,Z1] = np.meshgrid(np.arange(size[0]),np.arange(size[1]),np.arange(size[2]),indexing='ij')
            imgidx = np.stack([X1,Y1,Z1])
            imgidx = np.transpose(imgidx, axes = (1,2,3,0))
            idx11 = imgidx.reshape((-1,3))
            idx11 = idx11[mask_idx1, :]

            ## Adding Neighbors #######
            # idx11 = np.transpose(idx1FA.nonzero())
            lidx11 = idx11[:,:,None] + XYZ ## get coordinates of neighbors

            out_shape = tuple(i for i in Image_diff_norm.shape[0:3])

            y_val0 = np.zeros(out_shape)
            y_val1 = np.zeros(out_shape)
            y_val2 = np.zeros(out_shape)
            y_val3 = np.zeros(out_shape)
            y_val4 = np.zeros(out_shape)
            y_val5 = np.zeros(out_shape)
            yCount = np.zeros(out_shape) + 1e-20

            bsize = 100000
            outer_bsize = bsize*3

            for jj in range(0,lidx11.shape[0], outer_bsize):
              outer_upLim = min(jj+outer_bsize,lidx11.shape[0])
              outer_currSz = outer_upLim-jj
              ##########################################
              for jj0 in range(0,outer_currSz, bsize):
                j0 = jj + jj0
                upLim = min(jj0+bsize,outer_currSz)
                currSz = upLim-jj0
                X = np.zeros((currSz, lidx11.shape[2],Image_diff_norm.shape[3]))
                for j1 in range(currSz):
                  j = j1+j0
                  for k in range(lidx11.shape[2]):
                    coord = lidx11[j,:,k].reshape((3,))
                    X[j1,k,:] = Image_diff_norm[coord[0],coord[1],coord[2],:]
                    ##########################
                  # x1FA = Image_diff_norm[idx1FA,:]
                #X = X.transpose((0,2,1))
                XX = np.zeros((currSz, 128, 100))
                for i in range(X.shape[0]):
                  XX[i, :X.shape[1], :X.shape[2]] = X[i, :, :]
                  XX[i, X.shape[1]:, :X.shape[2]] = bvec.T
                [yval0, yval1, yval2, yval3, yval4, yval5]= model.predict(XX,batch_size=2000) #batch_size=5
                for j1 in range(currSz):
                  j = j1+j0
                  for k in range(lidx11.shape[2]):
                    coord = lidx11[j,:,k].reshape((3,))
                    y_val0[coord[0],coord[1],coord[2]] += yval0[j1, k]
                    y_val1[coord[0],coord[1],coord[2]] += yval1[j1, k]
                    y_val2[coord[0],coord[1],coord[2]] += yval2[j1, k]
                    y_val3[coord[0],coord[1],coord[2]] += yval3[j1, k]
                    y_val4[coord[0],coord[1],coord[2]] += yval4[j1, k]
                    y_val5[coord[0],coord[1],coord[2]] += yval5[j1, k]
                    yCount[coord[0],coord[1],coord[2]] += 1
              ######## Create Subprocess ###############################################
              # q = Queue()
              # q.put([jj,outer_currSz,bsize,lidx11,Image_diff_norm,bvec,y_val0,y_val1,y_val2,y_val3,y_val4,y_val5,yCount])
              # p = Process(target=tf_subprocess, args=(q,))
              # p.start()
              # p.join(timeout=300)
              # [y_val0,y_val1,y_val2,y_val3,y_val4,y_val5,yCount] = q.get()
              # q.close()
              ########################################################################


            y_val0 = (y_val0 / yCount) * mask
            y_val1 = (y_val1 / yCount) * mask
            y_val2 = (y_val2 / yCount) * mask
            y_val3 = (y_val3 / yCount) * mask
            y_val4 = (y_val4 / yCount) * mask
            y_val5 = (y_val5 / yCount) * mask

            y_val0 = y_val0[2:size[0]-2, 2:size[1]-2, 2:size[2]-2]
            y_val1 = y_val1[2:size[0]-2, 2:size[1]-2, 2:size[2]-2]
            y_val2 = y_val2[2:size[0]-2, 2:size[1]-2, 2:size[2]-2]
            y_val3 = y_val3[2:size[0]-2, 2:size[1]-2, 2:size[2]-2]
            y_val4 = y_val4[2:size[0]-2, 2:size[1]-2, 2:size[2]-2]
            y_val5 = y_val5[2:size[0]-2, 2:size[1]-2, 2:size[2]-2]

            tensor_val = np.stack((y_val0, y_val1, y_val2, y_val3, y_val4, y_val5), axis = 3)

            # Reconstruct quantetative parameter back from tensor_val
            D_est = dti.from_lower_triangular(tensor_val)
            eigvals,eigvecs = dti.decompose_tensor(D_est, min_diffusivity=0)
            tensor_fa = dti.fractional_anisotropy(eigvals)
            tensor_md = dti.mean_diffusivity(eigvals)
            tensor_rd = dti.radial_diffusivity(eigvals)
            tensor_ad = dti.axial_diffusivity(eigvals, axis=-1)
            OutputFolder = '/content/drive/MyDrive/Ananya_Singhal_2010110087/ICIP - classification/AD_data_21diff/pred_AD_results_21diff/Training_Patch5x5x5_Predict_FA_MD_RD_AD_volume_{}_Diff_for_SwinTransformer_5layer_ADNI_21_diff_additional'.format(diff_count)
            # OutputFolder = '/content/drive/MyDrive/Ananya_Singhal_2010110087/21st_feb_Patch5x5x5_Predict_FA_MD_RD_AD_volume_{}_Diff_for_SwinTransformer_5layer_ADNI_21_diff_based_on_ver1'.format(diff_count)
            # OutputFolder = '/media/snu/CT_DATA/cse_aiml_gdrive/AbhishekTiwari_at326/HCP_Young Adult_Data_Project/Patch3x3x3_Predict_FA_MD_AD_volume_90_Diff_for_SwinTransformer_3layer_Patch3x3x3_HCP_Youth_Adult_data_90_diff_64_diff_32_diff_21_diff_5_diff_upto10.tf'
            if not os.path.exists(OutputFolder):
              os.makedirs(os.path.join(OutputFolder,'FA'))
              os.makedirs(os.path.join(OutputFolder,'MD'))
              os.makedirs(os.path.join(OutputFolder,'RD'))
              os.makedirs(os.path.join(OutputFolder,'AD'))

              os.makedirs(os.path.join(OutputFolder,'img0'))
              os.makedirs(os.path.join(OutputFolder,'img1'))
              os.makedirs(os.path.join(OutputFolder,'img2'))
              os.makedirs(os.path.join(OutputFolder,'img3'))
              os.makedirs(os.path.join(OutputFolder,'img4'))
              os.makedirs(os.path.join(OutputFolder,'img5'))


            fileFA = os.path.join(OutputFolder,'FA','FA_{}.nii.gz'.format(lst))
            fileMD = os.path.join(OutputFolder,'MD','MD_{}.nii.gz'.format(lst))
            fileRD = os.path.join(OutputFolder,'RD','RD_{}.nii.gz'.format(lst))
            fileAD = os.path.join(OutputFolder,'AD','AD_,{}.nii.gz'.format(lst))

            file_img0 = os.path.join(OutputFolder,'img0','img0_{}.nii.gz'.format(lst))
            file_img1 = os.path.join(OutputFolder,'img1','img1_{}.nii.gz'.format(lst))
            file_img2 = os.path.join(OutputFolder,'img2','img2_{}.nii.gz'.format(lst))
            file_img3 = os.path.join(OutputFolder,'img3','img3_{}.nii.gz'.format(lst))
            file_img4 = os.path.join(OutputFolder,'img4','img4_{}.nii.gz'.format(lst))
            file_img5 = os.path.join(OutputFolder,'img5','img5_{}.nii.gz'.format(lst))

            save_nifti(fileFA, tensor_fa.astype(np.float32), affine)
            save_nifti(fileMD, tensor_md.astype(np.float32), affine)
            save_nifti(fileRD, tensor_rd.astype(np.float32), affine)
            save_nifti(fileAD, tensor_ad.astype(np.float32), affine)

            save_nifti(file_img0, y_val0.astype(np.float32), affine)
            save_nifti(file_img1, y_val1.astype(np.float32), affine)
            save_nifti(file_img2, y_val2.astype(np.float32), affine)
            save_nifti(file_img3, y_val3.astype(np.float32), affine)
            save_nifti(file_img4, y_val4.astype(np.float32), affine)
            save_nifti(file_img5, y_val5.astype(np.float32), affine)
            # save_nifti(os.path.join(fullpath,'tensor_rd.nii.gz'), dti_RD.astype(np.float32), affine)

(256, 256, 59, 22)
(22, 3)
(21, 3) (21,)
(256, 256, 59, 21)
(260, 260, 63, 21)
15/15 [==============================] - 3s 133ms/step
(256, 256, 59, 22)
(22, 3)
(21, 3) (21,)
(256, 256, 59, 21)
(260, 260, 63, 21)
15/15 [==============================] - 2s 125ms/step
(256, 256, 59, 22)
(22, 3)
(21, 3) (21,)
(256, 256, 59, 21)
(260, 260, 63, 21)
25/25 [==============================] - 3s 131ms/step
(256, 256, 59, 22)
(22, 3)
(21, 3) (21,)
(256, 256, 59, 21)
(260, 260, 63, 21)
35/35 [==============================] - 5s 130ms/step
